Setup Energy System Handler

In [52]:
# this is based on the demo from Edwin on 11-04-2024

from esdl import esdl
from esdl.esdl_handler import EnergySystemHandler
import pandas as pd
import numpy as np
import numpy_financial as npf

esh = EnergySystemHandler()
esh.load_file("TNVDW_cost_single_units_v1.esdl") #load here the file created in the mapeditor

energy_system = esh.get_energy_system()
instance_list = energy_system.instance
my_instance = instance_list[0]



Get input data from ESDL

In [53]:
#https://pyesdl.readthedocs.io/en/latest/Tutorials/tutorial4.html

asset_types_list = []
asset_names_list = []
asset_ids_list = []
asset_powers_list = []
asset_efficiencies_list = []
asset_investment_costs_list = [] #Capex, fixed_om and var_om are a singlevalue, not a range!
asset_fixed_opex_list = []
asset_var_opex_list = []


asset_parameters = pd.DataFrame()

In [54]:

#iterate through all ESDL elements: get all instances of type
for esdl_element in energy_system.eAllContents():

    #check if the element is an EnergyAsset
    if isinstance(esdl_element, esdl.EnergyAsset):

        #if it is, write its type, ID and name to a corresponding list
        asset_types_list.append(esdl_element.eClass.name)
        asset_ids_list.append(esdl_element.id)
        asset_names_list.append(esdl_element.name)

        #if it has costinformation, then append
        if esdl_element.costInformation is not None:
            asset_investment_costs_list.append(esdl_element.costInformation.investmentCosts.value) #singlevalues, not a range!
            asset_fixed_opex_list.append(esdl_element.costInformation.fixedOperationalAndMaintenanceCosts.value)
            asset_var_opex_list.append(esdl_element.costInformation.variableOperationalAndMaintenanceCosts.value)   
        else:
            asset_investment_costs_list.append("")
            asset_fixed_opex_list.append("")
            asset_var_opex_list.append("")


        #if an element is a producer, consumer or conversion, write its power
        if isinstance(esdl_element, esdl.Producer) or isinstance(esdl_element, esdl.Consumer) or isinstance(esdl_element, esdl.Conversion):
            asset_powers_list.append(esdl_element.power)
        else:
            asset_powers_list.append("")

        #if an element is an electrolyzer, write its efficiency
        if isinstance(esdl_element, esdl.Electrolyzer):
            asset_efficiencies_list.append(esdl_element.efficiency)
        else:
            asset_efficiencies_list.append("")

 

In [55]:


asset_parameters["Type"] = asset_types_list
asset_parameters["ID"] = asset_ids_list
asset_parameters["Name"] = asset_names_list
asset_parameters["Power"] = asset_powers_list
asset_parameters["Efficiency"] = asset_efficiencies_list
asset_parameters["Investment_costs"] = asset_investment_costs_list
asset_parameters["Fixed_opex"] = asset_fixed_opex_list
asset_parameters["Variable_opex"] = asset_var_opex_list

#decided to remove these from dataframe
asset_parameters = asset_parameters.drop("Type", axis=1)
asset_parameters = asset_parameters.drop("ID", axis=1)

#display dataframe (only works in jupiter notebook)
asset_parameters



,Name,Power,Efficiency,Investment_costs,Fixed_opex,Variable_opex
0,TNVDW,700000000.0,,1750.0,2.25,5.0
1,Electrolyzer,500.0,0.6,2000.0,2.0,0.0
2,Offtaker,24900000.0,,,,
3,ElectricityCable,,,,,
4,H2-pipe,,,,,


Data from Excel

General data OWF 

In [56]:
#start with defining the general information for the windfarm

duration_construction = 3
owf_duration_operation = 25
duration_decommissioning = 2

tender_year = 2027 #year at which business case is 0, i.e. year before start construction
year_construction_start = tender_year + 1
year_start_operation = year_construction_start + duration_construction
year_decommissioning_start = year_start_operation + owf_duration_operation


business_case_years = np.array(range(0,31))
calendar_year_list = list(range(tender_year, tender_year + 1 + duration_construction + owf_duration_operation + duration_decommissioning))  
#year_decommissioning_end+1 because range is up to year_decommissioning_end, so +1 is up to and including year_decommissioning_end


construction_years_list = list(range(year_construction_start, year_construction_start + duration_construction))
operations_years_list = list(range(year_start_operation, year_start_operation + owf_duration_operation))
decomissioning_years_list = list(range(year_decommissioning_start, year_decommissioning_start + duration_decommissioning))


Inflation - WACC data OWF

In [57]:
# get cost data from inflation-WACC sheet

owf_income_tax_rate = 0.258 #25.8%
owf_inflation = 0.02 #2%
owf_general_WACC = 0.085 #8.5%
owf_loan_interest_rate = 0.05 #5%
owf_length_of_loan = 15 # years
owf_loan_type = 'annuity'
owf_depreciation = 25 # years

Cost data OWF

In [58]:
# define cost data that is not included in Mapeditor

owf_contingency = 0.1 #10%
owf_loan_percentage = 0.75 #75%
owf_decomissioning = 0.02 # 2%

# get cost data from Mapeditor
owf_capex = asset_parameters['Investment_costs'][0] # in MEUR
owf_fixed_opex = asset_parameters['Fixed_opex'][0]/100  #converted 2 percent to 0.02
owf_var_opex = asset_parameters['Variable_opex'][0] # in Eur/MWh


In [59]:
# calcualte last cost data

owf_loan = owf_capex * owf_loan_percentage
owf_annuity_loan = -npf.pmt(owf_loan_interest_rate, owf_length_of_loan, owf_capex*owf_loan_percentage, fv=0, when='end')


PPA OWF

In [60]:
#PPA data for OWF

sold_to_electrolyser = 2849543 #MWh/year in 'used input data' Excel this is linked to the draft data input EYE document
sold_to_grid = 195458 #MWh/year in 'used input data' Excel this is linked to the draft data input EYE document
owf_revenues_to_electrolyser = 158.83 #MEUR in 'used input data' Excel this is linked to the draft data input EYE document
owf_revenues_to_market = 5.19 #MEUR in 'used input data' Excel this is linked to the draft data input EYE document

Start business case analysis

In [61]:
'''This is something I am still working on at the moment, so can be ignored for now!'''

# for now, the variables consist of the variables that were used in the sensitivity analysis for the OWF business case

# all variables = capex, fixed_opex, var_opex, revenues_to_electrolyser, revenues_to_market, income_tax_rate, inflation, general_wacc, loan_percentage, loan_interest_rate, duration_operation

# these are all the inputs for the owf basecase
input_owf_basecase = pd.Series([owf_capex, owf_fixed_opex, owf_var_opex, owf_revenues_to_electrolyser, owf_revenues_to_market, owf_income_tax_rate, owf_inflation, owf_general_WACC, owf_loan_percentage, owf_loan_interest_rate, owf_duration_operation])


In [62]:
# ignore for now
input_owf_basecase

0     1750.0000
1        0.0225
2        5.0000
3      158.8300
4        5.1900
5        0.2580
6        0.0200
7        0.0850
8        0.7500
9        0.0500
10      25.0000
dtype: float64

Construction phase

In [63]:
def construction_phase():
    ''' This function creates a dataframe that contains all cashflows in the construction phase'''

    # set up df
    df_construction_phase = pd.DataFrame(columns=range(31))
    df_construction_phase.columns = calendar_year_list
    df_construction_phase.columns.name = "construction_phase"
    
    #### function to calculate CAPEX
    def calculate_capex():
        ''' This function calculates the yearly capex for each of the construction years'''
        capex_list = []
    
        for x in calendar_year_list: 
            if x in construction_years_list:
                capex_list.append(-owf_capex/duration_construction)
            else: 
                capex_list.append(0)

        return capex_list
    
    # add CAPEX to df
    df_construction_phase.loc['capex'] = calculate_capex()

    # calculate total cashflow from investments then add to df
    # at this moment only one cashflow is in construction phase, can be expanded later
    total_cashflow_investment = calculate_capex()
    df_construction_phase.loc['total_cashflow_investment'] = total_cashflow_investment
    
    return df_construction_phase

In [64]:
construction_phase().style.format(precision=2)

construction_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
capex,0.00,-583.33,-583.33,-583.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
total_cashflow_investment,0.00,-583.33,-583.33,-583.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Operational phase

In [65]:
def operational_phase():
    ''' This function creates a dataframe that contains all cashflows in the operational phase'''
    
    #construct df
    df_operational_phase = pd.DataFrame(columns=range(31))
    df_operational_phase.columns = calendar_year_list
    df_operational_phase.columns.name = "operational_phase"
    
    #### function
    def fixed_opex():
        ''' This function calculates the fixed opex for all the operational years'''    

        fixed_opex_list = []

        # i,x in enumerate(list) will give an index to all elements in the list. 
        # i can then be used instead of list.index[x]
        for i, x in enumerate(calendar_year_list): 
            if x in operations_years_list:
                fixed_opex_list.append(-owf_fixed_opex*owf_capex*(1 + owf_inflation)**i)
            else: 
                fixed_opex_list.append(0)

        return fixed_opex_list
    
    # add fixed opex to df
    df_operational_phase.loc['fixed_opex'] = fixed_opex()

    #### function 
    def variable_opex():
        ''' This function calculates the variable opex for all the operational years'''
        
        variable_opex_list = []

        for i, x in enumerate(calendar_year_list):
            if x in operations_years_list:
                variable_opex_list.append(-owf_var_opex/1E6*(sold_to_electrolyser+sold_to_grid)*(1 + owf_inflation)**i)
            else: 
                variable_opex_list.append(0)

        return variable_opex_list   

    # add variable opex to df
    df_operational_phase.loc['variable_opex'] = variable_opex()

    # calculate total outflow opex and add to df
    total_outflow_opex = np.array(fixed_opex()) + np.array(variable_opex())
    df_operational_phase.loc['total_outflow_opex'] = total_outflow_opex

    #### function
    def revenues_electricity_ppa():
        ''' This function calculates the electricity revenues from the PPA for all the operational years'''

        revenues_electricity_ppa_list = []

        for i, x in enumerate(calendar_year_list):
            if x in operations_years_list:
                revenues_electricity_ppa_list.append(owf_revenues_to_electrolyser*(1 + owf_inflation)**i)
            else: 
                revenues_electricity_ppa_list.append(0)

        return revenues_electricity_ppa_list  
    
    # add electricity revenues from PPA to df
    df_operational_phase.loc['revenues_electricity_ppa'] = revenues_electricity_ppa()

    #### function 
    def revenues_electricity_market():
        ''' This function calculates the electricity revenues to market for all the operationl years'''

        revenues_electricity_market_list = []

        for i, x in enumerate(calendar_year_list):
            if x in operations_years_list:
                revenues_electricity_market_list.append(owf_revenues_to_market*(1 + owf_inflation)**i)
            else: 
                revenues_electricity_market_list.append(0)

        return revenues_electricity_market_list   

    # add electricity revenues from market to df
    df_operational_phase.loc['revenues_electricity_market'] = revenues_electricity_market()

    # calculate total revnues and add to df 
    total_revenues_opex = np.array(revenues_electricity_ppa()) + np.array(revenues_electricity_market())
    df_operational_phase.loc['total_revenues_opex'] = total_revenues_opex

    # calculate net cashflow from operation (=EBITDA) and add to df
    net_cashflow_operation = total_outflow_opex + total_revenues_opex
    df_operational_phase.loc['net_cashflow_operations'] = net_cashflow_operation
    
    return df_operational_phase

In [66]:
operational_phase().style.format(precision=2)

operational_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
fixed_opex,0.00,0.00,0.00,0.00,-42.62,-43.47,-44.34,-45.23,-46.13,-47.06,-48.00,-48.96,-49.94,-50.94,-51.95,-52.99,-54.05,-55.13,-56.24,-57.36,-58.51,-59.68,-60.87,-62.09,-63.33,-64.60,-65.89,-67.21,-68.55,0.00,0.00
variable_opex,0.00,0.00,0.00,0.00,-16.48,-16.81,-17.15,-17.49,-17.84,-18.20,-18.56,-18.93,-19.31,-19.70,-20.09,-20.49,-20.90,-21.32,-21.75,-22.18,-22.62,-23.08,-23.54,-24.01,-24.49,-24.98,-25.48,-25.99,-26.51,0.00,0.00
total_outflow_opex,0.00,0.00,0.00,0.00,-59.10,-60.28,-61.49,-62.72,-63.97,-65.25,-66.56,-67.89,-69.25,-70.63,-72.04,-73.48,-74.95,-76.45,-77.98,-79.54,-81.13,-82.76,-84.41,-86.10,-87.82,-89.58,-91.37,-93.20,-95.06,0.00,0.00
revenues_electricity_ppa,0.00,0.00,0.00,0.00,171.92,175.36,178.87,182.45,186.09,189.82,193.61,197.49,201.43,205.46,209.57,213.76,218.04,222.40,226.85,231.39,236.01,240.73,245.55,250.46,255.47,260.58,265.79,271.10,276.53,0.00,0.00
revenues_electricity_market,0.00,0.00,0.00,0.00,5.62,5.73,5.84,5.96,6.08,6.20,6.33,6.45,6.58,6.71,6.85,6.99,7.12,7.27,7.41,7.56,7.71,7.87,8.02,8.18,8.35,8.51,8.69,8.86,9.04,0.00,0.00
total_revenues_opex,0.00,0.00,0.00,0.00,177.54,181.09,184.71,188.41,192.18,196.02,199.94,203.94,208.02,212.18,216.42,220.75,225.16,229.67,234.26,238.95,243.73,248.60,253.57,258.64,263.82,269.09,274.47,279.96,285.56,0.00,0.00
net_cashflow_operations,0.00,0.00,0.00,0.00,118.44,120.81,123.22,125.69,128.20,130.77,133.38,136.05,138.77,141.55,144.38,147.26,150.21,153.21,156.28,159.40,162.59,165.84,169.16,172.54,176.00,179.52,183.11,186.77,190.50,0.00,0.00


Decomissioning phase

In [67]:
def decommissioning_phase():
    ''' This function creates a dataframe that contains all cashflows in the decommissioning phase'''
    
    #set up df
    df_decommissioning_phase = pd.DataFrame(columns=range(31))
    df_decommissioning_phase.columns = calendar_year_list
    df_decommissioning_phase.columns.name = "decommissioning_phase"
    
    #### function 
    def decommissioning_cost():
        ''' This function calculated the decommissioning costs for all the decommissioning years '''

        decommissioning_cost_list = []

        for i, x in enumerate(calendar_year_list):
            if x in decomissioning_years_list:
                decommissioning_cost_list.append(-owf_capex*owf_decomissioning*(1+owf_inflation)**i)
            else: 
                decommissioning_cost_list.append(0)

        return decommissioning_cost_list  
    
    # add decommissioning costs to df
    df_decommissioning_phase.loc['decommissioning_cost'] = decommissioning_cost()

    #add a row for the total decommissioning cashflows. Includes at this moment only one decommissioning cost, but could include more at a later stage.
    df_decommissioning_phase.loc['total_cashflow_decommissioning'] = decommissioning_cost() 
    
    
    return df_decommissioning_phase

In [68]:
decommissioning_phase().style.format(precision=2)

decommissioning_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
decommissioning_cost,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40
total_cashflow_decommissioning,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40


Taxes & profits - part 1

In [69]:
def taxes_and_profits_part1():
    ''' This function creates a dataframe that contains all cashflows for taxes & profits
        It consists of two parts, as it is dependent on the debt and loan calculations'''

    # construct a df
    df_taxes_profits = pd.DataFrame(columns=range(31))
    df_taxes_profits.columns = calendar_year_list
    df_taxes_profits.columns.name = "taxes_and_profits"
    

    #### function
    def depreciation():
        ''' This function calculates the depreciation for the capex investment during the operational years'''

        depreciation_list = []

        for x in calendar_year_list:
            if x in operations_years_list:
                depreciation_list.append(-owf_capex/owf_depreciation)
            else: 
                depreciation_list.append(0)

        return depreciation_list  
    
    #### function 
    def ebit():
        ''' This function calculates the ebit by adding depreciation (which is negative) to ebitda (net cashflow operations)'''

        net_cashflow_operations = operational_phase().loc['net_cashflow_operations'].tolist()
        
        ebit_list = np.array(net_cashflow_operations) + np.array(depreciation())

        return ebit_list

    # add depreciation and ebit to df
    df_taxes_profits.loc['owf_depreciation'] = depreciation()
    df_taxes_profits.loc['ebit'] = ebit()

    
    return df_taxes_profits

In [70]:
taxes_and_profits_part1().style.format(precision=2)

taxes_and_profits,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
owf_depreciation,0.00,0.00,0.00,0.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,0.00,0.00
ebit,0.00,0.00,0.00,0.00,48.44,50.81,53.22,55.69,58.20,60.77,63.38,66.05,68.77,71.55,74.38,77.26,80.21,83.21,86.28,89.40,92.59,95.84,99.16,102.54,106.00,109.52,113.11,116.77,120.50,0.00,0.00


In [71]:
# now we need the interest costs, but these are in Excel linked to dept&loan.
# so first we will work on the debt & loan part, later continue with: interest costs, ebt, tax expenses, net profits


Debt and loan - part 1

In [72]:
def debt_and_loan_part1():
    ''' This function creates a dataframe that contains all cashflows for debt and loans
        It consists of two parts as it is dependent on the taxes & profits calculations'''

    # construct df
    row_names = [
    "begin_of_year",
    "drawdown",
    "repayment_capital",
    "end_of_year",
    "repayment_interest",]

    df_debt_and_loan = pd.DataFrame(0.0, index=row_names, columns=calendar_year_list)
    df_debt_and_loan.columns.name = "debt_and_loan"

    #### function
    def debt_drawdown():
        ''' This function calculates the amount that is lent for the capex investments for each of the consturction years'''   

        drawdown_list = []

        for x in calendar_year_list:
            if x in construction_years_list:
                drawdown_list.append(owf_capex*owf_loan_percentage/duration_construction)
            else: 
                drawdown_list.append(0)

        return drawdown_list  

    # add debt drawdown to df
    df_debt_and_loan.loc["drawdown"] = debt_drawdown()

    #### function
    def debt_calculations():
        ''' This function calculates debt at the begin of the year, debt at the end of the year, 
            repayment of interest and repayment of capital. 
            These are dependent on each other and therefore combined into one function'''


        for x in calendar_year_list:
        
            #begin of year
            if x == tender_year:
                df_debt_and_loan[x]['begin_of_year'] = 0
            else:
                df_debt_and_loan[x]['begin_of_year'] = df_debt_and_loan[x-1]['end_of_year']
        
            #repayment interest
            if x in operations_years_list:
                df_debt_and_loan[x]['repayment_interest'] = -(df_debt_and_loan[x]['begin_of_year'] + df_debt_and_loan[x]['drawdown']) * owf_loan_interest_rate

            else:
                df_debt_and_loan[x]['repayment_interest'] = 0

            #repayment capital
            if x in operations_years_list:
                df_debt_and_loan[x]['repayment_capital'] = -min(df_debt_and_loan[x]['begin_of_year'] + df_debt_and_loan[x]['drawdown'], owf_annuity_loan + df_debt_and_loan[x]['repayment_interest'])

            else: 
                df_debt_and_loan[x]['repayment_capital'] = 0
        
            #end of year
            df_debt_and_loan[x]['end_of_year'] = df_debt_and_loan[x]['begin_of_year'] + df_debt_and_loan[x]['drawdown'] + df_debt_and_loan[x]['repayment_capital']

        return df_debt_and_loan


    return debt_calculations()
    

In [73]:
debt_and_loan_part1().style.format(precision=2)

debt_and_loan,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
begin_of_year,0.00,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
drawdown,0.00,437.50,437.50,437.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_capital,0.00,0.00,0.00,0.00,-60.82,-63.87,-67.06,-70.41,-73.93,-77.63,-81.51,-85.59,-89.87,-94.36,-99.08,-104.03,-109.23,-114.69,-120.43,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
end_of_year,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_interest,0.00,0.00,0.00,0.00,-65.62,-62.58,-59.39,-56.04,-52.52,-48.82,-44.94,-40.86,-36.58,-32.09,-27.37,-22.42,-17.22,-11.76,-6.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00


In [74]:
# For the cash flow available for debt service, we need the tax expenses. So here we go back to Taxes & profits, and after that is finished we go back to Debt & loan - part 2

Taxes & profits - part 2

In [75]:
# continue with: interest costs, ebt, tax expenses, net profits

In [76]:
def taxes_and_profits_part2():
    ''' This function creates a dataframe that contains all cashflows for taxes and profits
        This is the second part of the taxes and profits calculations.'''

    # load the dataframe that was constructed in part 1 of the taxes and profits calculations
    df_taxes_profits = taxes_and_profits_part1()

    # the interest costs were calculated in the debt_and_loan_part1 function
    # they are imported into this dataframe because they are used to calculate ebt from ebit
    interest_costs = debt_and_loan_part1().values[4]
    df_taxes_profits.loc['interest_costs'] = interest_costs

    # the ebt is calculated by adding the ebit to the interest costs (which is negative) and added to the df
    ebit = df_taxes_profits.loc['ebit'].tolist()
    ebt = ebit + interest_costs
    df_taxes_profits.loc['ebt'] = ebt

    #### function
    def tax_expenses():
        '''This function calculates the taxes that have to be paid if ebt is positive'''

        tax_expenses_list = []

        for x in calendar_year_list:
            if ebt[calendar_year_list.index(x)] > 0:
                tax_expenses_list.append(-ebt[calendar_year_list.index(x)] * owf_income_tax_rate)
            else:
                tax_expenses_list.append(0)

        return tax_expenses_list
    
    # add tax expenses to df 
    df_taxes_profits.loc['tax_expenses'] = tax_expenses()

    # calculate net profits by adding tax expenses to ebt and add to df
    net_profits =  ebt + np.array(tax_expenses())
    df_taxes_profits.loc['net_profits'] = net_profits

    return df_taxes_profits

In [77]:
taxes_and_profits_part2().style.format(precision=2)

taxes_and_profits,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
owf_depreciation,0.00,0.00,0.00,0.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,0.00,0.00
ebit,0.00,0.00,0.00,0.00,48.44,50.81,53.22,55.69,58.20,60.77,63.38,66.05,68.77,71.55,74.38,77.26,80.21,83.21,86.28,89.40,92.59,95.84,99.16,102.54,106.00,109.52,113.11,116.77,120.50,0.00,0.00
interest_costs,0.00,0.00,0.00,0.00,-65.62,-62.58,-59.39,-56.04,-52.52,-48.82,-44.94,-40.86,-36.58,-32.09,-27.37,-22.42,-17.22,-11.76,-6.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
ebt,0.00,0.00,0.00,0.00,-17.19,-11.78,-6.17,-0.35,5.69,11.95,18.44,25.19,32.19,39.46,47.00,54.85,62.99,71.46,80.26,89.40,92.59,95.84,99.16,102.54,106.00,109.52,113.11,116.77,120.50,0.00,0.00
tax_expenses,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.47,-3.08,-4.76,-6.50,-8.30,-10.18,-12.13,-14.15,-16.25,-18.44,-20.71,-23.07,-23.89,-24.73,-25.58,-26.46,-27.35,-28.25,-29.18,-30.13,-31.09,0.00,0.00
net_profits,0.00,0.00,0.00,0.00,-17.19,-11.78,-6.17,-0.35,4.22,8.86,13.69,18.69,23.88,29.28,34.88,40.70,46.74,53.02,59.55,66.34,68.70,71.12,73.58,76.09,78.65,81.26,83.92,86.64,89.41,0.00,0.00


Debt & loan - part 2

In [78]:
# still to do for debt & loan: 

#cash flow available for debt service
#debt service (capital + interest)
#cash flow after debt service


In [79]:
def debt_and_loan_part2():
    ''' This function creates a dataframe that contains all cashflows for debt and loans
        This is the second part of the debt and loans calculations.'''

    # inport the df from debt_and_loan_part1 to continue with the same df
    df_debt_and_loan = debt_and_loan_part1()

    # the tax expenses were calculated in taxes_and_profits_part2 and are imported here
    # the cashflow available for debt is calculated by adding tax expenses to the net cashflow of operations
    # add cashflow for debt to df
    tax_expenses = taxes_and_profits_part2().loc['tax_expenses'].tolist()
    net_cashflow_operations = operational_phase().loc['net_cashflow_operations'].tolist()
    
    cash_flow_for_debt = np.array(tax_expenses) + net_cashflow_operations
    df_debt_and_loan.loc['cash_flow_for_debt'] = cash_flow_for_debt

    #### function
    def debt_service():
        ''' This function calculates debt service (i.e. capital and interest repayments)'''
        debt_service_list = []

        for x in calendar_year_list: 
            debt_service_list.append(df_debt_and_loan[x]['repayment_capital'] + df_debt_and_loan[x]['repayment_interest'])

        return debt_service_list
    
    # add debt service to df
    df_debt_and_loan.loc['debt_service'] = debt_service()

    # calculate cash after debt service and add it to the df
    cash_after_debt_service = cash_flow_for_debt + debt_service()
    df_debt_and_loan.loc['cash_after_debt_service'] = cash_after_debt_service
    
    return df_debt_and_loan

In [80]:
debt_and_loan_part2().style.format(precision=2)

debt_and_loan,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
begin_of_year,0.00,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
drawdown,0.00,437.50,437.50,437.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_capital,0.00,0.00,0.00,0.00,-60.82,-63.87,-67.06,-70.41,-73.93,-77.63,-81.51,-85.59,-89.87,-94.36,-99.08,-104.03,-109.23,-114.69,-120.43,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
end_of_year,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_interest,0.00,0.00,0.00,0.00,-65.62,-62.58,-59.39,-56.04,-52.52,-48.82,-44.94,-40.86,-36.58,-32.09,-27.37,-22.42,-17.22,-11.76,-6.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
cash_flow_for_debt,0.00,0.00,0.00,0.00,118.44,120.81,123.22,125.69,126.74,127.68,128.62,129.55,130.47,131.37,132.25,133.11,133.96,134.78,135.57,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,0.00
debt_service,0.00,0.00,0.00,0.00,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
cash_after_debt_service,0.00,0.00,0.00,0.00,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,0.00


Project reserves

In [81]:
def project_reserves():
    '''This function creates a dataframe that contains all cashflows for project reserves (i.e., contingency) '''

    #construct df
    column_names = [
    "contingency_injection",
    "contingency_reserve_balance",
    "contingency_reserve_to_dividents",]

    df_project_reserves = pd.DataFrame(0.0, index=column_names, columns=calendar_year_list)
    df_project_reserves.columns.name = "project_reserves"

    #### function 
    def contingency_injection():
        '''This function calculates the contingency required for the project'''
        contingency_injection_list = []

        for x in calendar_year_list:
            if calendar_year_list.index(x) == 0:
                contingency_injection_list.append(-owf_capex*owf_contingency)
        
            else:
                contingency_injection_list.append(0)

        return contingency_injection_list

    # add contingency injection to df
    df_project_reserves.loc['contingency_injection'] = contingency_injection()


    #### function to calculate contingency reserve balance and contingency reserve to dividents
    def contingency_project():
        ''' This function calculates the balance of contingency for each year 
            and the amount that is left over in the last year that is going to dividents'''

        for x in calendar_year_list:

            #contingency_reserve_balance
            if x == tender_year:
                df_project_reserves[x]['contingency_reserve_balance'] = -df_project_reserves[x]['contingency_injection']
            else: 
                df_project_reserves[x]['contingency_reserve_balance'] = df_project_reserves[x-1]['contingency_reserve_balance'] - df_project_reserves[x]['contingency_injection'] - df_project_reserves[x]['contingency_reserve_to_dividents']

            #contingency_reserve_to_dividents"
            if x == calendar_year_list[-1] and df_project_reserves[x]['contingency_reserve_balance'] > 0:
                df_project_reserves[x]['contingency_reserve_to_dividents'] = df_project_reserves[x]['contingency_reserve_balance']
            else:
                df_project_reserves[x]['contingency_reserve_to_dividents'] = 0

        return df_project_reserves


    return contingency_project()

In [82]:
project_reserves().style.format(precision=2)

project_reserves,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
contingency_injection,-175.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
contingency_reserve_balance,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00
contingency_reserve_to_dividents,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,175.00


Equity funding

In [83]:
def equity_funding():
    '''This function creates a dataframe that contains all cashflows for equity funding '''

    # construct df
    df_equity_funding = pd.DataFrame(columns=range(31))
    df_equity_funding.columns = calendar_year_list
    df_equity_funding.columns.name = "equity_funding"
    
    # get input data required for calculations
    contingency_injection = project_reserves().loc['contingency_injection']
    total_cashflow_decommissioning = decommissioning_phase().loc['total_cashflow_decommissioning']
    cash_after_debt_service = debt_and_loan_part2().loc['cash_after_debt_service']
    capex = construction_phase().loc['capex']
    contingency_reserve_to_dividents = np.array(project_reserves().loc['contingency_reserve_to_dividents'])

    # calculate equity injection and add to df
    equity_injection = contingency_injection + (capex *(1-owf_loan_percentage)) + total_cashflow_decommissioning
    df_equity_funding.loc['equity_injection'] = equity_injection

    # calculate dividents results and add to df
    dividents_results = cash_after_debt_service + contingency_reserve_to_dividents
    df_equity_funding.loc['dividents_results'] = dividents_results

    # calculate equity cash flow result and add to df
    equity_cash_flow_result = equity_injection + dividents_results 
    df_equity_funding.loc['equity_cash_flow_result'] = equity_cash_flow_result

    return df_equity_funding

In [84]:
equity_funding().style.format(precision=2)

equity_funding,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
equity_injection,-175.00,-145.83,-145.83,-145.83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40
dividents_results,0.00,0.00,0.00,0.00,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,175.00
equity_cash_flow_result,-175.00,-145.83,-145.83,-145.83,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,-62.15,111.60


Display all cashflows

In [85]:
# Display all individual dataframes

display(construction_phase().style.format(precision=2))
display(operational_phase().style.format(precision=2))
display(decommissioning_phase().style.format(precision=2))
display(taxes_and_profits_part2().style.format(precision=2))
display(debt_and_loan_part2().style.format(precision=2))
display(project_reserves().style.format(precision=2))
display(equity_funding().style.format(precision=2))

construction_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
capex,0.00,-583.33,-583.33,-583.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
total_cashflow_investment,0.00,-583.33,-583.33,-583.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


operational_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
fixed_opex,0.00,0.00,0.00,0.00,-42.62,-43.47,-44.34,-45.23,-46.13,-47.06,-48.00,-48.96,-49.94,-50.94,-51.95,-52.99,-54.05,-55.13,-56.24,-57.36,-58.51,-59.68,-60.87,-62.09,-63.33,-64.60,-65.89,-67.21,-68.55,0.00,0.00
variable_opex,0.00,0.00,0.00,0.00,-16.48,-16.81,-17.15,-17.49,-17.84,-18.20,-18.56,-18.93,-19.31,-19.70,-20.09,-20.49,-20.90,-21.32,-21.75,-22.18,-22.62,-23.08,-23.54,-24.01,-24.49,-24.98,-25.48,-25.99,-26.51,0.00,0.00
total_outflow_opex,0.00,0.00,0.00,0.00,-59.10,-60.28,-61.49,-62.72,-63.97,-65.25,-66.56,-67.89,-69.25,-70.63,-72.04,-73.48,-74.95,-76.45,-77.98,-79.54,-81.13,-82.76,-84.41,-86.10,-87.82,-89.58,-91.37,-93.20,-95.06,0.00,0.00
revenues_electricity_ppa,0.00,0.00,0.00,0.00,171.92,175.36,178.87,182.45,186.09,189.82,193.61,197.49,201.43,205.46,209.57,213.76,218.04,222.40,226.85,231.39,236.01,240.73,245.55,250.46,255.47,260.58,265.79,271.10,276.53,0.00,0.00
revenues_electricity_market,0.00,0.00,0.00,0.00,5.62,5.73,5.84,5.96,6.08,6.20,6.33,6.45,6.58,6.71,6.85,6.99,7.12,7.27,7.41,7.56,7.71,7.87,8.02,8.18,8.35,8.51,8.69,8.86,9.04,0.00,0.00
total_revenues_opex,0.00,0.00,0.00,0.00,177.54,181.09,184.71,188.41,192.18,196.02,199.94,203.94,208.02,212.18,216.42,220.75,225.16,229.67,234.26,238.95,243.73,248.60,253.57,258.64,263.82,269.09,274.47,279.96,285.56,0.00,0.00
net_cashflow_operations,0.00,0.00,0.00,0.00,118.44,120.81,123.22,125.69,128.20,130.77,133.38,136.05,138.77,141.55,144.38,147.26,150.21,153.21,156.28,159.40,162.59,165.84,169.16,172.54,176.00,179.52,183.11,186.77,190.50,0.00,0.00


decommissioning_phase,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
decommissioning_cost,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40
total_cashflow_decommissioning,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40


taxes_and_profits,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
owf_depreciation,0.00,0.00,0.00,0.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,-70.00,0.00,0.00
ebit,0.00,0.00,0.00,0.00,48.44,50.81,53.22,55.69,58.20,60.77,63.38,66.05,68.77,71.55,74.38,77.26,80.21,83.21,86.28,89.40,92.59,95.84,99.16,102.54,106.00,109.52,113.11,116.77,120.50,0.00,0.00
interest_costs,0.00,0.00,0.00,0.00,-65.62,-62.58,-59.39,-56.04,-52.52,-48.82,-44.94,-40.86,-36.58,-32.09,-27.37,-22.42,-17.22,-11.76,-6.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
ebt,0.00,0.00,0.00,0.00,-17.19,-11.78,-6.17,-0.35,5.69,11.95,18.44,25.19,32.19,39.46,47.00,54.85,62.99,71.46,80.26,89.40,92.59,95.84,99.16,102.54,106.00,109.52,113.11,116.77,120.50,0.00,0.00
tax_expenses,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.47,-3.08,-4.76,-6.50,-8.30,-10.18,-12.13,-14.15,-16.25,-18.44,-20.71,-23.07,-23.89,-24.73,-25.58,-26.46,-27.35,-28.25,-29.18,-30.13,-31.09,0.00,0.00
net_profits,0.00,0.00,0.00,0.00,-17.19,-11.78,-6.17,-0.35,4.22,8.86,13.69,18.69,23.88,29.28,34.88,40.70,46.74,53.02,59.55,66.34,68.70,71.12,73.58,76.09,78.65,81.26,83.92,86.64,89.41,0.00,0.00


debt_and_loan,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
begin_of_year,0.00,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
drawdown,0.00,437.50,437.50,437.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_capital,0.00,0.00,0.00,0.00,-60.82,-63.87,-67.06,-70.41,-73.93,-77.63,-81.51,-85.59,-89.87,-94.36,-99.08,-104.03,-109.23,-114.69,-120.43,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
end_of_year,0.00,437.50,875.00,1312.50,1251.68,1187.81,1120.75,1050.34,976.41,898.78,817.27,731.68,641.82,547.46,448.38,344.35,235.12,120.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
repayment_interest,0.00,0.00,0.00,0.00,-65.62,-62.58,-59.39,-56.04,-52.52,-48.82,-44.94,-40.86,-36.58,-32.09,-27.37,-22.42,-17.22,-11.76,-6.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
cash_flow_for_debt,0.00,0.00,0.00,0.00,118.44,120.81,123.22,125.69,126.74,127.68,128.62,129.55,130.47,131.37,132.25,133.11,133.96,134.78,135.57,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,0.00
debt_service,0.00,0.00,0.00,0.00,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-126.45,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
cash_after_debt_service,0.00,0.00,0.00,0.00,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,0.00


project_reserves,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
contingency_injection,-175.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
contingency_reserve_balance,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00,175.00
contingency_reserve_to_dividents,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,175.00


equity_funding,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
equity_injection,-175.00,-145.83,-145.83,-145.83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-62.15,-63.40
dividents_results,0.00,0.00,0.00,0.00,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,0.00,175.00
equity_cash_flow_result,-175.00,-145.83,-145.83,-145.83,-8.01,-5.64,-3.22,-0.76,0.29,1.24,2.17,3.10,4.02,4.92,5.80,6.67,7.51,8.33,9.12,136.34,138.70,141.12,143.58,146.09,148.65,151.26,153.92,156.64,159.41,-62.15,111.60


In [86]:
# construct large df including all rows

In [87]:
all_dfs = [construction_phase(), operational_phase(), decommissioning_phase(), taxes_and_profits_part2(), debt_and_loan_part2(), project_reserves(), equity_funding()]
full_df = pd.concat(all_dfs)
# full_df.style.format(precision=2)   #shows the full df

Present value of cash flows

In [88]:
def present_value_cashflows():
    ''' This function creates a dataframe that contains all present value cashflows '''

    # construct df
    df_present_value = pd.DataFrame(columns=range(31))
    df_present_value.columns = calendar_year_list
    df_present_value.columns.name = "present_value"
    
    # calculate sum of the net project cashflows and add to df
    total_cashflow_investment = construction_phase().loc['total_cashflow_investment']
    net_cashflow_operations = operational_phase().loc['net_cashflow_operations']
    total_cashflow_decommissioning = decommissioning_phase().loc['total_cashflow_decommissioning']
    
    sum_net_project_cash_flows = total_cashflow_investment + net_cashflow_operations + total_cashflow_decommissioning
    df_present_value.loc['sum_net_project_cash_flows'] = sum_net_project_cash_flows
    
    #### function
    def present_value_net_cashflows():
        '''This function calculates the present value of the net cashflows'''

        present_value_net_cashflows_list = []

        for i, x in enumerate(calendar_year_list):
        
            present_value_net_cashflows_list.append(sum_net_project_cash_flows.tolist()[i] / (1 + owf_general_WACC) ** i)

        return present_value_net_cashflows_list
    
    # add present value of net cashflows to df
    df_present_value.loc['present_value_net_cashflows'] = present_value_net_cashflows()

    #### function
    def cumulative_value_net_cashflows():
        '''This function calculates the cumulative value of the net cashflows'''
        cumulative_value_net_cashflows_list = []

        for i, x in enumerate(calendar_year_list):
            if x == tender_year:
                cumulative_value_net_cashflows_list.append(present_value_net_cashflows()[i])
            else:
                cumulative_value_net_cashflows_list.append(present_value_net_cashflows()[i] + cumulative_value_net_cashflows_list[i-1])

        return cumulative_value_net_cashflows_list

    # add cumulative value of net cashflows to df
    df_present_value.loc['cumulative_value_net_cashflows'] = cumulative_value_net_cashflows()

    #### function 
    def present_value_equity_cashflows():
        '''This function calculates the present value of equity cashflows'''

        present_value_equity_cashflows_list = []
        equity_cash_flow_result = equity_funding().loc['equity_cash_flow_result'].tolist()

        for i, x in enumerate(calendar_year_list): 
            present_value_equity_cashflows_list.append(equity_cash_flow_result[i] / (1 + owf_general_WACC) ** i)

        return present_value_equity_cashflows_list

    # add present value of equity cashflows to df
    df_present_value.loc['present_value_equity_cashflows'] = present_value_equity_cashflows()

    return df_present_value

In [89]:
present_value_cashflows().style.format(precision=2)

present_value,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
sum_net_project_cash_flows,0.00,-583.33,-583.33,-583.33,118.44,120.81,123.22,125.69,128.20,130.77,133.38,136.05,138.77,141.55,144.38,147.26,150.21,153.21,156.28,159.40,162.59,165.84,169.16,172.54,176.00,179.52,183.11,186.77,190.50,-62.15,-63.40
present_value_net_cashflows,0.00,-537.63,-495.52,-456.70,85.46,80.34,75.53,71.01,66.75,62.75,58.99,55.46,52.14,49.01,46.08,43.32,40.72,38.28,35.99,33.83,31.81,29.90,28.11,26.43,24.84,23.35,21.95,20.64,19.40,-5.83,-5.49
cumulative_value_net_cashflows,0.00,-537.63,-1033.15,-1489.85,-1404.38,-1324.04,-1248.51,-1177.51,-1110.75,-1048.00,-989.01,-933.55,-881.41,-832.40,-786.32,-743.01,-702.28,-664.00,-628.01,-594.18,-562.38,-532.48,-504.37,-477.94,-453.10,-429.75,-407.79,-387.15,-367.75,-373.58,-379.07
present_value_equity_cashflows,-175.00,-134.41,-123.88,-114.17,-5.78,-3.75,-1.98,-0.43,0.15,0.59,0.96,1.26,1.51,1.70,1.85,1.96,2.04,2.08,2.10,28.94,27.13,25.44,23.86,22.37,20.98,19.68,18.46,17.31,16.24,-5.83,9.66


Cumulative equity and debt cash flows

In [90]:
def cumulative_equity_and_debt_cashflows():
    '''This function creates a dataframe that contains the cumulative equity and debt cashflows'''

    # construct df
    df_cumulative_equity_and_debt = pd.DataFrame(columns=range(31))
    df_cumulative_equity_and_debt.columns = calendar_year_list
    df_cumulative_equity_and_debt.columns.name = "cumulative equity and debt"
    
    ### function 
    def cumulative_equity_cashflow():
        ''' This function calculates the cumulative equity cashflow'''

        cumulative_equity_cashflow_list = []
        equity_cash_flow_result = equity_funding().loc['equity_cash_flow_result'].tolist()

        for i, x in enumerate(calendar_year_list): 
            if x == tender_year:
                cumulative_equity_cashflow_list.append(equity_cash_flow_result[i])

            else:
                cumulative_equity_cashflow_list.append(equity_cash_flow_result[i] + cumulative_equity_cashflow_list[i-1])

        return cumulative_equity_cashflow_list

    # add cumulative equity cashflow to df
    df_cumulative_equity_and_debt.loc['cumulative_equity_cashflow'] = cumulative_equity_cashflow()

    #### function to calculate cumulative debt cashflow
    def cumulative_debt_cashflow():
        '''This function calculates the cumulative debt cashflow'''

        cumulative_debt_cashflow_list = []

        for x in calendar_year_list:
            cumulative_debt_cashflow_list.append(-debt_and_loan_part1()[x]['end_of_year'])

        return cumulative_debt_cashflow_list
    
    # add cumulative debt cashflow to df
    df_cumulative_equity_and_debt.loc['cumulative_debt_cashflow'] = cumulative_debt_cashflow()
    
    return df_cumulative_equity_and_debt.style.format(precision=2)   

In [91]:
cumulative_equity_and_debt_cashflows()

cumulative equity and debt,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
cumulative_equity_cashflow,-175.00,-320.83,-466.67,-612.50,-620.51,-626.15,-629.37,-630.13,-629.85,-628.61,-626.44,-623.34,-619.32,-614.40,-608.60,-601.93,-594.42,-586.10,-576.97,-440.63,-301.93,-160.81,-17.24,128.85,277.50,428.76,582.68,739.32,898.74,836.58,948.19
cumulative_debt_cashflow,-0.00,-437.50,-875.00,-1312.50,-1251.68,-1187.81,-1120.75,-1050.34,-976.41,-898.78,-817.27,-731.68,-641.82,-547.46,-448.38,-344.35,-235.12,-120.43,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00


Discounted cash flows for levelized cost

In [92]:
def discounted_cashflows_for_levelized_cost():
    '''This function creates a dataframe that contains the discounted cashflows for levelized cost calculations'''

    # construct df
    df_discounted_cashflows_for_levelized_cost = pd.DataFrame(columns=range(31))
    df_discounted_cashflows_for_levelized_cost.columns = calendar_year_list
    df_discounted_cashflows_for_levelized_cost.columns.name = "discounted cashflows for levelized cost"
    
    # set up variables for calculations
    capex = construction_phase().loc['capex']
    
    time = pd.Series(range(31), index=capex.index) 
    discount_term = (1 + owf_general_WACC) ** time

    # calculate dicounted capex and add to df
    df_discounted_cashflows_for_levelized_cost.loc['capex'] = capex / discount_term

    # calculate discounted fixed opex and add to df
    fixed_opex = operational_phase().loc['fixed_opex']
    df_discounted_cashflows_for_levelized_cost.loc['fixed_opex'] = fixed_opex / discount_term

    # calculate discounted variable opex and add to df
    variable_opex = operational_phase().loc['variable_opex']
    df_discounted_cashflows_for_levelized_cost.loc['variable_opex'] = variable_opex / discount_term

    # calculate discounted decomissioning cost
    decommissioning_cost = decommissioning_phase().loc['decommissioning_cost']
    df_discounted_cashflows_for_levelized_cost.loc['decommissioning'] = decommissioning_cost / discount_term

    # calculate discounted interest costs
    interest_costs = taxes_and_profits_part2().loc['interest_costs']
    df_discounted_cashflows_for_levelized_cost.loc['interest_costs'] = interest_costs / discount_term

    # calculate discounted contingency injection
    contingency_injection = project_reserves().loc['contingency_injection']
    contingency_to_dividents = project_reserves().loc['contingency_reserve_to_dividents']
    df_discounted_cashflows_for_levelized_cost.loc['contingency'] = (contingency_injection + contingency_to_dividents) / discount_term

    # calculate discounted tax expenses
    tax_expenses = taxes_and_profits_part2().loc['tax_expenses']
    df_discounted_cashflows_for_levelized_cost.loc['tax_expenses'] = tax_expenses / discount_term

    # calculate discounted revenues ppa
    revenues_ppa = operational_phase().loc['revenues_electricity_ppa']
    df_discounted_cashflows_for_levelized_cost.loc['revenues_ppa'] = revenues_ppa / discount_term

    # calculate discounted reveues market
    revenues_market = operational_phase().loc['revenues_electricity_market']
    df_discounted_cashflows_for_levelized_cost.loc['revenues_market'] = revenues_market / discount_term
    
    #### function   
    def total_electricity_produced(): 
        '''This function discounts the total electricity produced during the operational years'''

        total_electricity_produced_list = []

        for i, x in enumerate(calendar_year_list):
            if x in operations_years_list:
                total_electricity_produced_list.append((sold_to_electrolyser + sold_to_grid) / (1 + owf_general_WACC) ** i)

            else: 
                total_electricity_produced_list.append(0)
        
        return total_electricity_produced_list

    # add total electricity produced to df
    df_discounted_cashflows_for_levelized_cost.loc['total_electricity_produced'] = total_electricity_produced()
    
    return df_discounted_cashflows_for_levelized_cost



In [93]:
discounted_cashflows_for_levelized_cost().style.format(precision=2)

discounted cashflows for levelized cost,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
capex,0.00,-537.63,-495.52,-456.70,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
fixed_opex,0.00,0.00,0.00,0.00,-30.75,-28.91,-27.18,-25.55,-24.02,-22.58,-21.23,-19.96,-18.76,-17.64,-16.58,-15.59,-14.65,-13.78,-12.95,-12.17,-11.45,-10.76,-10.12,-9.51,-8.94,-8.40,-7.90,-7.43,-6.98,0.00,0.00
variable_opex,0.00,0.00,0.00,0.00,-11.89,-11.18,-10.51,-9.88,-9.29,-8.73,-8.21,-7.72,-7.25,-6.82,-6.41,-6.03,-5.67,-5.33,-5.01,-4.71,-4.43,-4.16,-3.91,-3.68,-3.46,-3.25,-3.05,-2.87,-2.70,0.00,0.00
decommissioning,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-5.83,-5.49
interest_costs,0.00,0.00,0.00,0.00,-47.35,-41.62,-36.40,-31.66,-27.34,-23.43,-19.88,-16.66,-13.74,-11.11,-8.74,-6.59,-4.67,-2.94,-1.39,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
contingency,-175.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.14
tax_expenses,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.76,-1.48,-2.10,-2.65,-3.12,-3.52,-3.87,-4.16,-4.41,-4.61,-4.77,-4.90,-4.67,-4.46,-4.25,-4.05,-3.86,-3.68,-3.50,-3.33,-3.17,0.00,0.00
revenues_ppa,0.00,0.00,0.00,0.00,124.05,116.62,109.64,103.07,96.89,91.09,85.63,80.50,75.68,71.15,66.88,62.88,59.11,55.57,52.24,49.11,46.17,43.40,40.80,38.36,36.06,33.90,31.87,29.96,28.16,0.00,0.00
revenues_market,0.00,0.00,0.00,0.00,4.05,3.81,3.58,3.37,3.17,2.98,2.80,2.63,2.47,2.32,2.19,2.05,1.93,1.82,1.71,1.60,1.51,1.42,1.33,1.25,1.18,1.11,1.04,0.98,0.92,0.00,0.00
total_electricity_produced,0.00,0.00,0.00,0.00,2197194.42,2025063.98,1866418.41,1720201.30,1585438.99,1461234.09,1346759.53,1241253.02,1144012.00,1054388.94,971787.04,895656.26,825489.64,760819.95,701216.54,646282.53,595652.10,548988.11,505979.82,466340.85,429807.24,396135.70,365102.03,336499.57,310137.85,0.00,0.00


Levelized costs

In [94]:
def levelized_cost_and_revenues():
    ''' This function creates a dataframe that contains the levelized costs and revenues'''

    # construct df
    row_names_levelized_cost = discounted_cashflows_for_levelized_cost().index.tolist()
    row_names_levelized_cost.pop()

    df_levelized_cost = pd.DataFrame(0.0, index=row_names_levelized_cost, columns=['cost','revenues'])
    df_levelized_cost.columns.name = "levelized cost calculations"

    #### function
    def levelized_cost():
        '''This function calculates the levelized costs and revenues'''

        total_electricity_produced_sum = discounted_cashflows_for_levelized_cost().loc['total_electricity_produced'].sum()

        for x in row_names_levelized_cost:

            if discounted_cashflows_for_levelized_cost().loc[x].sum() > 0:
                df_levelized_cost.loc[x]['revenues'] = discounted_cashflows_for_levelized_cost().loc[x].sum() * 1E6 / total_electricity_produced_sum
            else:
                df_levelized_cost.loc[x]['cost'] = -discounted_cashflows_for_levelized_cost().loc[x].sum() * 1E6 / total_electricity_produced_sum

        return df_levelized_cost

    # update df to that obtained from function
    df_levelized_cost = levelized_cost()

    #### function
    def profits():
        '''This function calculates the profits'''

        if df_levelized_cost['cost'].sum() < df_levelized_cost['revenues'].sum():
            profits = df_levelized_cost['revenues'].sum() - df_levelized_cost['cost'].sum()

        else:
            profits = 0

        return profits
    
    # add profits to df
    df_levelized_cost.loc['profits'] = [0,0]
    df_levelized_cost.loc['profits']['cost'] = profits()

    #### function   
    def unprofitable_gap():
        '''This function calculates the unprofitable gap'''

        if df_levelized_cost['cost'].sum() > df_levelized_cost['revenues'].sum():
            gap = df_levelized_cost['cost'].sum() - df_levelized_cost['revenues'].sum()

        else:
            gap = 0

        return gap
    
    # add unprofitable gap to df
    df_levelized_cost.loc['unprofitable_gap'] = [0,0]
    df_levelized_cost.loc['unprofitable_gap']['revenues'] = unprofitable_gap()


    return df_levelized_cost


In [95]:
levelized_cost_and_revenues().style.format(precision=2)

levelized cost calculations,cost,revenues
capex,61.06,0.00
fixed_opex,16.55,0.00
variable_opex,6.40,0.00
decommissioning,0.46,0.00
interest_costs,12.03,0.00
contingency,6.55,0.00
tax_expenses,3.09,0.00
revenues_ppa,0.00,66.76
revenues_market,0.00,2.18
profits,0.00,0.00


Project KPI's

In [96]:
def project_kpi():
    '''This function creates a dataframe that contains the project KPI's'''

    # create df
    df_project_kpi = pd.DataFrame(columns=['Value','Unit'])
    df_project_kpi.columns.name = "Project KPIs"
    
    # get input data
    sum_net_project_cashflows = present_value_cashflows().loc['sum_net_project_cash_flows'].tolist()
    net_profits = taxes_and_profits_part2().loc['net_profits'].tolist()
    total_cashflow_investment = construction_phase().loc['total_cashflow_investment'].tolist()
    present_value_net_cashflows = present_value_cashflows().loc['present_value_net_cashflows'].tolist()
    
    # calculate NPV and add to df
    net_present_value = [npf.npv(owf_general_WACC,sum_net_project_cashflows), 'MEUR']
    df_project_kpi.loc['net_present_value'] = net_present_value

    # calculate IRR and add to df
    internal_rate_of_return = [npf.irr(sum_net_project_cashflows) * 100,'%']    # timex 100 to give percentage
    df_project_kpi.loc['internal_rate_of_return'] = internal_rate_of_return

    # calculate ROI and add to df
    return_on_investment = [np.array(net_profits).sum() / -np.array(total_cashflow_investment).sum() *100,'%']
    df_project_kpi.loc['return_on_investment'] = return_on_investment
    
    # calculate PP and add to df
    payback_period = [-np.array(total_cashflow_investment).sum() / (np.array(net_profits).sum() / owf_duration_operation), 'years']
    df_project_kpi.loc['payback_period'] = payback_period


    ##### function
    def discounted_return_on_investment():
        ''' This function calculates the discounted return on investment'''

        pv_investment = 0
        pv_returns = 0

        for i, x in enumerate(calendar_year_list):
            if x in construction_years_list:
                pv_investment = pv_investment + present_value_net_cashflows[i]

            elif x in operations_years_list:
                pv_returns = pv_returns + present_value_net_cashflows[i]
    
        discounted_roi_list = [(pv_investment + pv_returns) / -pv_investment * 100,'%']     # times 100 to convert to percentage

        return discounted_roi_list
    
    # add discounted ROI to df
    df_project_kpi.loc['discounted_return_on_investment'] = discounted_return_on_investment()

    #### function
    def discounted_payback_period():
        '''This function calculates the discounted payback period'''

        pv_investment = 0
        pv_returns = 0

        for i, x in enumerate(calendar_year_list):
            if x in construction_years_list:
                pv_investment = pv_investment + present_value_net_cashflows[i]

            elif x in operations_years_list:
                pv_returns = pv_returns + present_value_net_cashflows[i]


        discounted_pp_list = [-pv_investment / (pv_returns / owf_duration_operation),'years']
    
        return discounted_pp_list


    #### add discounted payback period to df
    df_project_kpi.loc['discounted_payback_period'] = discounted_payback_period()

    
    return df_project_kpi.style.format(precision=2)

In [97]:
project_kpi()

Project KPIs,Value,Unit
net_present_value,-379.07,MEUR
internal_rate_of_return,5.73,%
return_on_investment,61.36,%
payback_period,40.75,years
discounted_return_on_investment,-24.68,%
discounted_payback_period,33.19,years


Equity KPI's

In [98]:
def equity_kpi():
    '''This function creates a dataframe that contains the equity KPI's'''

    # construct df
    df_equity_kpi = pd.DataFrame(columns=['Value','Unit'])
    df_equity_kpi.columns.name = "Equity KPIs"
    
    # get input data
    equity_injection = equity_funding().loc['equity_injection'].tolist()
    dividents_results = equity_funding().loc['dividents_results'].tolist()
    equity_cash_flow_result = equity_funding().loc['equity_cash_flow_result'].tolist()
    present_value_equity_cashflows = present_value_cashflows().loc['present_value_equity_cashflows'].tolist()
    
    # calculate NPV and add to df
    net_present_value = [npf.npv(owf_general_WACC,equity_cash_flow_result), 'MEUR']
    df_equity_kpi.loc['net_present_value'] = net_present_value

    # calculate IRR and add to df
    internal_rate_of_return = [npf.irr(equity_cash_flow_result) * 100,'%']    # timex 100 to give percentage
    df_equity_kpi.loc['internal_rate_of_return'] = internal_rate_of_return

    # calculate ROI and add to df
    return_on_investment = [np.array(dividents_results).sum() / -np.array(equity_injection).sum() *100,'%']
    df_equity_kpi.loc['return_of_investment'] = return_on_investment

    # calculate PP and add to df
    payback_period = [-np.array(equity_injection).sum() / (np.array(dividents_results).sum() / owf_duration_operation), 'years']
    df_equity_kpi.loc['payback_period'] = payback_period
    
    
    ##### function
    def discounted_return_on_investment():
        '''This function calculates the discounted return on investment'''

        pv_investment = 0
        pv_returns = 0

        for i, x in enumerate(calendar_year_list):
            if x in construction_years_list:
                pv_investment = pv_investment + present_value_equity_cashflows[i]

            elif x in operations_years_list:
                pv_returns = pv_returns + present_value_equity_cashflows[i]
    
        discounted_roi_list = [(pv_investment + pv_returns) / -pv_investment * 100,'%']     # times 100 to convert to percentage

        return discounted_roi_list
    
    # add discounted ROI to df
    df_equity_kpi.loc['discounted_return_on_investment'] = discounted_return_on_investment()

    #### function
    def discounted_payback_period():
        '''This function calculates the discounted payback period'''

        pv_investment = 0
        pv_returns = 0

        for i, x in enumerate(calendar_year_list):
            if x in construction_years_list:
                pv_investment = pv_investment + present_value_equity_cashflows[i]

            elif x in operations_years_list:
                pv_returns = pv_returns + present_value_equity_cashflows[i]


        discounted_pp_list = [-pv_investment / (pv_returns / owf_duration_operation),'years']
    
        return discounted_pp_list


    # add discounted payback period to df
    df_equity_kpi.loc['discounted_payback_period'] = discounted_payback_period()
    
    return df_equity_kpi.style.format(precision=2)


In [99]:
equity_kpi()

Equity KPIs,Value,Unit
net_present_value,-318.96,MEUR
internal_rate_of_return,4.30,%
return_of_investment,228.47,%
payback_period,10.94,years
discounted_return_on_investment,-39.68,%
discounted_payback_period,41.44,years


Output KPI's

In [100]:
def output_kpi():
    '''This function creates a dataframe that contains the output KPI's'''

    # construct df
    df_output_kpi = pd.DataFrame(columns=['Value','Unit'])
    df_output_kpi.columns.name = "Output KPIs"
    
    # calculate levelized costs and add to df
    levelized_costs = [levelized_cost_and_revenues()['cost'][:7].sum(),'Eur/MWh']
    df_output_kpi.loc['levelized_cost'] = levelized_costs

    # calculate levelized revenues and add to df
    levelized_revenues = [levelized_cost_and_revenues()['revenues'][7:9].sum(),'Eur/MWh']
    df_output_kpi.loc['levelized_revenues'] = levelized_revenues

    # calculate levelized profits and add to df
    levelized_profits = [df_output_kpi['Value']['levelized_revenues'] - df_output_kpi['Value']['levelized_cost'],'Eur/MWh']
    df_output_kpi.loc['levelized_profits'] = levelized_profits

    return df_output_kpi.style.format(precision=2)

In [101]:
output_kpi()

Output KPIs,Value,Unit
levelized_cost,106.15,Eur/MWh
levelized_revenues,68.94,Eur/MWh
levelized_profits,-37.21,Eur/MWh


End of base case business case calculations